In [1]:
# using PyTorch 2.0.1
# using PyTorch 2.0.1
# using PyTorch 2.0.1


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel #, AutoModelForMaskedLM

import pickle
from tqdm import tqdm

# import pickle
# import re
import os
import csv
import warnings
warnings.filterwarnings('ignore')


%run hyena_utility.py

%run preprocess_utility.py

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/scratch/local/48896213/ipykernel_826563/3242630166.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/sunhuaikuan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

### Load Human Chrom Sequences from .fa File

In [10]:
fasta_file = "/blue/xiaofan/sunhuaikuan/gpn/examples/msa/genome.hg38rg.fa"
chrom_sequences = read_fasta(fasta_file)

def get_subsequence(chrom_name, start_pos, length):
    
    # print(chrom_name, start_pos, length)
    if chrom_name in chrom_sequences:
        sequence = chrom_sequences[chrom_name]
        subsequence = sequence[start_pos:start_pos + length]
        return subsequence
    else:
        raise ValueError(f"Chromosome '{chrom_name}' not found in the FASTA file.")

In [3]:
# def Cur_Inference(model, tokenizer, max_length, device, sequence):
def Cur_Inference(model, sequence):  # model, tokenizer, max_length, device, 

    '''
    this selects which backbone to use, and grabs weights/ config from HF
    4 options:
      'hyenadna-tiny-1k-seqlen'   # fine-tune on colab ok
      'hyenadna-small-32k-seqlen'
      'hyenadna-medium-160k-seqlen'  # inference only on colab
      'hyenadna-medium-450k-seqlen'  # inference only on colab
      'hyenadna-large-1m-seqlen'  # inference only on colab
    '''

    # you only need to select which model to use here, we'll do the rest!
    # pretrained_model_name = 'hyenadna-small-32k-seqlen'

    # max_lengths = {
    #     'hyenadna-tiny-1k-seqlen': 1024,
    #     'hyenadna-small-32k-seqlen': 32768,
    #     'hyenadna-medium-160k-seqlen': 160000,
    #     'hyenadna-medium-450k-seqlen': 450000,  # T4 up to here
    #     'hyenadna-large-1m-seqlen': 1_000_000,  # only A100 (paid tier)
    # }


    #### Single embedding example ####

    # create a sample 450k long, prepare
    # sequence = 'ACTG' * int(max_length/4)
    tok_seq = tokenizer(sequence)
    tok_seq = tok_seq["input_ids"]  # grab ids

    # place on device, convert to tensor
    tok_seq = torch.LongTensor(tok_seq).unsqueeze(0)  # unsqueeze for batch dim
    tok_seq = tok_seq.to(device)

    # prep model and forward
    # model.to(device)
    # model.eval()
    with torch.inference_mode():
        embeddings = model(tok_seq)

    # cls_embedding = embeddings.last_hidden_state[:, 0, :]
    # cls_embedding = embeddings[:, 0, :]
    
    mean_embeddings = embeddings.mean(dim=1) # Mean across the sequence length dimension
    mean_embeddings = mean_embeddings.squeeze(0)  # This will change the shape to [256]

    
    # print(embeddings.shape)  # embeddings here!
    # return cls_embedding
    return mean_embeddings


In [4]:
# def Subsequence2Embedding(model, tokenizer, max_length, device, subsequence):
def Subsequence2Embedding(subsequence):
    # embeddings = My_Inference(model, tokenizer, max_length, device, subsequence)
    embeddings = Cur_Inference(model, subsequence)
    # print(embeddings.shape)
    # print (embeddings)
    # return embeddings[0,int(max_length/2),:]
    return embeddings  # embeddings[0,0,:]

# max_length

### Main Process

In [6]:
pretrained_model_name = 'hyenadna-small-32k-seqlen'
pretrained_model_name = 'hyenadna-medium-160k-seqlen'
# pretrained_model_name = 'hyenadna-medium-450k-seqlen'
# pretrained_model_name = 'hyenadna-large-1m-seqlen'
model, tokenizer, max_length =  get_model_tokenizer_maxlen(pretrained_model_name)
model.to(device)
model.eval()

Using device: cuda
Git LFS initialized.


Cloning into 'hyenadna-medium-160k-seqlen'...


Loaded pretrained weights ok!


HyenaDNAModel(
  (backbone): LMBackbone(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(16, 256)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): HyenaOperator(
          (dropout): Dropout(p=0.0, inplace=False)
          (in_proj): Linear(in_features=256, out_features=768, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
          (short_filter): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(2,), groups=768)
          (filter_fn): HyenaFilter(
            (dropout): Dropout(p=0.0, inplace=False)
            (pos_emb): PositionalEmbedding()
            (implicit_filter): Sequential(
              (0): Linear(in_features=5, out_features=64, bias=True)
              (1): Sin()
              (2): Linear(in_features=64, out_features=64, bias=True)
              (3): Sin()
              (4): Linear(in_features=64, out_features=64, bias=True)
              (5): Sin()
              (6): Linear(in_features=

In [17]:
# datafile='clinvar_20240805.noncoding'
# datafile='clinvar_20240805.missense_matched'
datafile='methylation'
# datafile='GSM6637962_CpG_coverage20_GRCh38.bed.gz'

In [25]:
data_filename = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task05-methylation/GSM6637962_CpG_coverage20_GRCh38.bed.gz' 
df = preprocess_datafile(data_filename)
df

,CHROM,START,SIZE,y
0,1,788407,128,100
1,1,788472,128,80
2,1,788825,128,95
3,1,788844,128,90
4,1,788859,128,100
...,...,...,...,...
54789,Y,56870903,128,65
54790,Y,56870983,128,17
54791,Y,56871109,128,34
54792,Y,56871502,128,0


In [19]:
# %%time

# comp = {'A':1, 'C':2, 'G':3, 'T':4}


# max_length= 128 # 186

# csv_Filename =datafile + '_with_embeddings.csv'
# if os.path.exists(csv_Filename):
#     os.remove(csv_Filename)


# df=pd.read_csv(datafile+'.txt', delimiter='\t')
# # df=pd.read_csv('clinvar_20240805.missense_matched.txt', delimiter='\t')

# columns_to_keep=['CHROM','POS','ID','REF','ALT','Pathogenicity']
# df = df[columns_to_keep]

# # Merge CHROM=9 and '9' etc

# for i in range(1,23):
#     df.loc[df['CHROM']==i,'CHROM']=str(i)


# df['START']= df['POS']- max_length //2 -1
# df['END']  = df['START'] + max_length
# df['SIZE'] = max_length

# df=df[~df['CHROM'].isna()]

# Pathogenicity_dict={'B':0,'P':1}
# df['y'] = df['Pathogenicity'].map(Pathogenicity_dict)
# # df['CHROM'].value_counts()
# df.drop(['POS','ID','Pathogenicity','END'], axis=1, inplace=True)
# df


In [28]:
%%time

csv_Filename =datafile + '_hyena_embedding.csv'

if os.path.exists(csv_Filename):
    os.remove(csv_Filename)



# column_names = [f'{i}' for i in range(1, df.shape[1])]
# column_names.extend([ 'y'])
# df_head = pd.DataFrame(columns=column_names)
# df_head.to_csv(csv_Filename)


rows=[]
for index, row in df.iterrows():      
    chrom=row['CHROM']
    # pos=row['POS']
    # pos_start=row['POS']-round(max_length / 2)
    pos_start=row['START']

    if pos_start<=1:
        pos_start=1
    # ref=row['REF']
    # alt=row['ALT']        
    y=row['y']
    length = row['SIZE'] # max_length
    
    subsequence = get_subsequence(chrom, pos_start, length)
    if 'N' in subsequence:
        print("The character 'N' is present in the string.")
        
    embedding = Subsequence2Embedding(subsequence)
    # print(embedding.shape)

    # feature=np.array(embedding_df.iloc[64])
    rows.append(np.append(embedding.cpu().numpy(),  [y])) # chrom,  length,  comp[ref],comp[alt],

    if index > 0 and (index % 5000) == 0:
        append_rows_to_csv(csv_Filename, rows)
        rows=[]
        print (f"index = {index} completed")
        
append_rows_to_csv(csv_Filename, rows)

print(f"Create File: "+csv_Filename)

index = 5000 completed
index = 10000 completed
index = 15000 completed
The character 'N' is present in the string.
The character 'N' is present in the string.
index = 20000 completed
The character 'N' is present in the string.
The character 'N' is present in the string.
The character 'N' is present in the string.
The character 'N' is present in the string.
index = 25000 completed
The character 'N' is present in the string.
The character 'N' is present in the string.
index = 30000 completed
The character 'N' is present in the string.
index = 35000 completed
index = 40000 completed
The character 'N' is present in the string.
The character 'N' is present in the string.
index = 45000 completed
Create File: methylation_hyena_embedding.csv
CPU times: user 5min 58s, sys: 163 ms, total: 5min 58s
Wall time: 6min 13s


### Load CSV File

In [29]:
# csv_Filename =datafile + '_hyena_embedding.csv'
df = load_embedding_file(csv_Filename)
df

,1,2,3,4,5,6,7,8,9,10,...,248,249,250,251,252,253,254,255,256,y
0,-0.162548,-0.813792,-0.502185,-0.825777,-0.877710,-0.901205,1.084889,-0.752337,-0.211693,0.480058,...,0.536883,0.422926,-1.036571,-1.505365,-1.542530,1.273687,0.541963,0.330082,-0.782260,80.0
1,0.545844,-0.499106,-0.211811,-0.605006,-0.363712,-0.115525,0.974813,-0.903288,-0.637590,0.212635,...,0.576334,0.362970,-0.771966,-1.256579,-1.035106,1.435116,0.312089,0.661820,-0.845981,95.0
2,0.607656,-0.510390,-0.237946,-0.559285,-0.386759,-0.082885,0.962740,-0.900656,-0.646205,0.154209,...,0.558357,0.384656,-0.830481,-1.283214,-1.037074,1.399000,0.265083,0.666149,-0.886538,90.0
3,0.522967,-0.544051,-0.275782,-0.638263,-0.471732,-0.228838,1.009440,-0.955511,-0.494366,0.158926,...,0.464206,0.379089,-0.885517,-1.393619,-1.167201,1.349000,0.287169,0.557183,-0.955191,100.0
4,0.547458,-0.516419,-0.256440,-0.622132,-0.409509,-0.208485,0.989845,-0.941072,-0.492761,0.190209,...,0.468198,0.383017,-0.850792,-1.364307,-1.135328,1.340600,0.339875,0.563152,-0.898098,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47947,-0.204415,-0.610424,-0.525940,-0.834108,-1.071689,-0.866407,1.295794,-0.787243,-0.093626,0.270392,...,0.469020,0.538909,-1.000904,-1.556431,-1.477009,1.182108,0.483010,0.468306,-1.007607,65.0
47948,-0.191999,-0.571596,-0.412285,-0.903216,-1.231649,-0.926968,1.293870,-0.727409,-0.031014,0.126409,...,0.491232,0.636665,-0.939041,-1.525331,-1.463316,1.163905,0.463123,0.561148,-1.014437,17.0
47949,-0.007314,-0.681754,-0.413436,-0.843044,-0.748679,-0.847264,1.296018,-0.867100,-0.041741,0.354103,...,0.579400,0.478063,-0.936292,-1.452830,-1.407930,1.272913,0.476933,0.465113,-0.959612,34.0
47950,-0.167239,-0.801911,-0.522504,-0.799539,-0.921501,-0.894758,1.127078,-0.804274,-0.197577,0.462455,...,0.476291,0.385570,-1.059132,-1.483167,-1.471666,1.242552,0.524794,0.329107,-0.814684,0.0
